<a href="https://colab.research.google.com/github/WB-Jang/Training-Evaluation-Data-Set-Construction-by-Crawling/blob/main/QeustionDecompDataSet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas
!pip install langchain_openai
!pip install langchain_google_genai
!pip install langchain_core

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.7/84.7 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 19.8 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 1.2.1
    Uninstalling langchain-core-1.2.1:
      Successfully uninstalled langchain-core-1.2.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.3/53.3 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 426.6/426.6 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.3/233.3 kB 11.1 MB/s eta 0:00:00
  Attempting uninstall: google-auth
    Found existing installation: google-auth 2.43.0
    Uninstalling google-auth-2.43.0:
      Successfully uninstalled google-auth-2.43.0
  Attempting uninstall: google-genai
    Found existing installation: google-genai 1.55.0
    Uninstalling google-genai-1.55.0:
      Successfully uninstalled goo

In [ ]:
import pandas as pd
import os
root = os.getcwd()
print('현재 경로 : ',root)

path = './drive/MyDrive/Data/Break-dataset/QDMR-high-level/train.csv'
if os.path.exists(path):
  print(f'{path} 경로가 존재합니다')
else:
  print('확인 필요')

현재 경로 :  /content
./drive/MyDrive/Data/Break-dataset/QDMR-high-level/train.csv 경로가 존재합니다


In [ ]:
import pandas as pd
import os

data = pd.read_csv('./drive/MyDrive/Data/Break-dataset/QDMR-high-level/train.csv',sep=',', encoding='utf-8-sig')

print('len(data) : ',len(data))
print('shape(data) : ', data.shape)

len(data) :  17503
shape(data) :  (17503, 5)


In [ ]:
data.head()

,question_id,question_text,decomposition,operators,split
0,CWQ_train_WebQTest-1_ffaa5abbcbb61976117b13d82...,"What office, also held by a member of the Main...",return office that was held by James K. Polk ...,"['select', 'filter']",train
1,CWQ_train_WebQTest-1001_ef4f8588342569743387a9...,What country is the composer of Up Out My Face...,return the composer of Up Out My Face ;return...,"['select', 'project']",train
2,CWQ_train_WebQTest-1003_15a2c52b25f9475f44be70...,Who was Pakistan's prime minister in 1999 and ...,return prime minister of Pakistan in 1999 ;re...,"['select', 'filter']",train
3,CWQ_train_WebQTest-1003_39835eae9754fe630428c0...,Who as Pakistan's Prime Minister in 1999 also ...,return Pakistan 's Prime Minister in 1999 ;re...,"['select', 'filter']",train
4,CWQ_train_WebQTest-1003_98d0a9f163aaad0e328d70...,Who held his governmental position after May 2...,return Prime Minister of Pakistan in 1999 ;re...,"['select', 'filter']",train


### Data Conversion Strategy
train.csv에 있는 Decompostion을 그대로 LangChain의 chain에 넣자
프롬프트는 아래와 같이 진행해보는 것으로



[[
System: You are an expert data annotator. Convert the given structured Question Decomposition (QDMR) into a natural language step-by-step plan.

Remove technical jargon like "return", "filter", "aggregate".

Replace references like "#1" with the actual content.

Make it sound like a human breaking down a problem.

Input (QDMR): return flights ;return #1 from washington ;return #2 to boston Output (Natural): > 1. Find all available flights. 2. Filter for flights departing from Washington. 3. Narrow down to flights arriving in Boston.
]]



In [ ]:
from google.colab import userdata
import os

# Secret에 저장된 키를 환경 변수로 설정
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

from langchain_openai import ChatOpenAI
gpt5mini_llm = ChatOpenAI(model="gpt-5-mini", temperature=0, streaming=True)

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')
gemini_25_flash_llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash-preview-09-2025",temperature=0)

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
prompt_trans = ChatPromptTemplate.from_messages([
    ("system", """
      You are an expert translator. Convert the given english Question into Korean Question.
      ***All output should be in Korean.***
      """),
    ("human", "[question]\\n{question}")
]
)
prompt_decomp = ChatPromptTemplate.from_messages([
    ("system", """
      You are an expert data annotator. Convert the given structured Question Decomposition (QDMR) into a natural language step-by-step plan.
      Remove technical jargon like "return", "filter", "aggregate".
      Replace references like "#1" with the actual content.
      Make it sound like a human breaking down a problem.

      Input (QDMR): return flights ;return #1 from washington ;return #2 to boston Output (Natural): > 1. Find all available flights. 2. Filter for flights departing from Washington. 3. Narrow down to flights arriving in Boston.
      ***All output should be in Korean.***
      """),
    ("human", "[input]\\n{input}")
]
)

# chain_trans = prompt_trans | gpt5mini_llm | StrOutputParser()
# chain_decomp = prompt_decomp | gpt5mini_llm | StrOutputParser()

chain_trans = prompt_trans | gemini_25_flash_llm | StrOutputParser()
chain_decomp = prompt_decomp | gemini_25_flash_llm | StrOutputParser()

# 아래는 GPT-4o에 넣고 돌린 결과

In [ ]:
trans_generation = chain_trans.invoke({"question": data.iloc[0,1]})
print(trans_generation)
decomp_generation = chain_decomp.invoke({"input": data.iloc[0,2]})
print(decomp_generation)

[질문]\n제임스 K. 포크가 대통령이 되기 전에 맡았던 직책은 무엇이며, 메인 하원의원도 맡고 있는 직책은 무엇입니까?
1. 제임스 K. 포크가 대통령이 되기 전에 맡았던 직책을 찾아보세요.  
2. 메인 하원의원이 맡았던 직책 중에서 제임스 K. 포크가 맡았던 직책을 확인하세요.


# 아래는 GPT-5-mini에 넣고 돌린 결과

In [ ]:
trans_generation = chain_trans.invoke({"question": data.iloc[0,1]})
print(trans_generation)
decomp_generation = chain_decomp.invoke({"input": data.iloc[0,2]})
print(decomp_generation)

[질문]\n제임스 K. 포크가 대통령이 되기 전에 맡았던 직책은 무엇이며, 메인 하원의원도 맡고 있던 직책은 무엇입니까?
1. 제임스 K. 포크가 대통령이 되기 전에 맡았던 직책을 찾아보세요. 2. 메인 하원의원이 맡았던 직책 중에서 제임스 K. 포크가 맡았던 직책을 확인하세요.


# 아래는 gemini-2.5-flash-preview-09-2025에 넣고 돌린 결과

In [ ]:
trans_generation = chain_trans.invoke({"question": data.iloc[0,1]})
print(trans_generation)
decomp_generation = chain_decomp.invoke({"input": data.iloc[0,2]})
print(decomp_generation)

제임스 K. 포크가 대통령이 되기 전에 메인 주 하원의원 중 한 명도 역임했던 직책은 무엇이었습니까?
> 1. 제임스 K. 폴크가 대통령이 되기 전에 어떤 직책들을 맡았었는지 알아보세요.
> 2. 1단계에서 찾은 직책들 중, 메인주 하원의회 의원도 맡았던 직책을 찾아보세요.


In [ ]:
cols = data.columns
data_decomposed = data[cols[:3]]

data_decomposed['question_kor']=''
data_decomposed['decomposition_new']=''
# data_decomposed.iloc[1,3]='test'
# data_decomposed.iloc[1,4]='test'
data_decomposed

/tmp/ipython-input-2277955072.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_decomposed['question_kor']=''


,question_id,question_text,decomposition,question_kor,decomposition_new
0,CWQ_train_WebQTest-1_ffaa5abbcbb61976117b13d82...,"What office, also held by a member of the Main...",return office that was held by James K. Polk ...,,
1,CWQ_train_WebQTest-1001_ef4f8588342569743387a9...,What country is the composer of Up Out My Face...,return the composer of Up Out My Face ;return...,,
2,CWQ_train_WebQTest-1003_15a2c52b25f9475f44be70...,Who was Pakistan's prime minister in 1999 and ...,return prime minister of Pakistan in 1999 ;re...,,
3,CWQ_train_WebQTest-1003_39835eae9754fe630428c0...,Who as Pakistan's Prime Minister in 1999 also ...,return Pakistan 's Prime Minister in 1999 ;re...,,
4,CWQ_train_WebQTest-1003_98d0a9f163aaad0e328d70...,Who held his governmental position after May 2...,return Prime Minister of Pakistan in 1999 ;re...,,
...,...,...,...,...,...
17498,HOTPOT_train_5ae82f9a55429952e35eaa80,The Knicks–Nuggets was the most penalized on-c...,return altercation that Knicks–Nuggets was th...,,
17499,HOTPOT_train_5ae830bb5542997ec2727757,When does the sit-in street protests opposed b...,return the sit-in street protests opposed by ...,,
17500,HOTPOT_train_5ae8343255429952e35eaa8e,"""Two Swords"" is the first episdoe of a televsi...",return season of the televsion series where t...,,
17501,HOTPOT_train_5ae8395f5542997ec272777c,The Wingate business hotel chain is operated b...,return the business hotel chain The Wingate ;r...,,


In [ ]:
def trans_decomp(data, i,j,k):
  for idx in range(i, j):
    trans_generation = chain_trans.invoke({"question": data.iloc[idx,1]})
    print(f'{idx}번째 질문 번역 결과 : {trans_generation}')
    data.iloc[idx,3] = trans_generation
    print('trans_success')
    decomp_generation = chain_decomp.invoke({"input": data.iloc[idx,2]})
    print(f'{idx}번째 질문 분해 결과 : {decomp_generation}')
    data.iloc[idx,4] = decomp_generation
    print('decomp_success')

  print(data.iloc[i:j,:])
  data.iloc[i:j,:].to_csv(f'./drive/MyDrive/Data/Break-dataset/QDMR-high-level/train_trans_decomp_{k}.csv',index=False)

In [ ]:
# trans_decomp(data=data_decomposed, k=1) #gpt-5-mini
# trans_decomp(data=data_decomposed, k=2) #gpt-5-mini
# trans_decomp(data=data_decomposed, k=3) #gemini-2.5-flash
# trans_decomp(data=data_decomposed, i=1400, j=1500, k=4) #gemini-2.5-flash
# trans_decomp(data=data_decomposed, i=1500, j=1700, k=5) #gemini-2.5-flash
# trans_decomp(data=data_decomposed, i=1700, j=1900, k=6) #gemini-2.5-flash
# trans_decomp(data=data_decomposed, i=1900, j=2200, k=7) #gemini-2.5-flash
# trans_decomp(data=data_decomposed, i=2200, j=2500, k=8) #gemini-2.5-flash
# trans_decomp(data=data_decomposed, i=2500, j=2900, k=9) #gemini-2.5-flash
# trans_decomp(data=data_decomposed, i=4000, j=4500, k=12) #gemini-2.5-flash
# trans_decomp(data=data_decomposed, i=4500, j=5500, k=13) #gemini-2.5-flash
# trans_decomp(data=data_decomposed, i=5500, j=6500, k=14) #gemini-2.5-flash
# trans_decomp(data=data_decomposed, i=6500, j=7500, k=15) #gemini-2.5-flash
# trans_decomp(data=data_decomposed, i=7500, j=9000, k=16) #gemini-2.5-flash
trans_decomp(data=data_decomposed, i=9000, j=10500, k=17) #gemini-2.5-flash

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
비슷한 방식으로 주어진 영어 질문을 한국어로 번역하세요:
비슷한 방식으로 주어진 영어 질문을 한국어로 번역하세요:
비슷한 방식으로 주어진 영어 질문을 한국어로 번역하세요:
비슷한 방식으로 주어진 영어 질문을 한국어로 번역하세요:
비슷한 방식으로 주어진 영어 질문을 한국어로 번역하세요:
비슷한 방식으로 주어진 영어 질문을 한국어로 번역하세요:
비슷한 방식으로 주어진 영어 질문을 한국어로 번역하세요:
비슷한 방식으로 주어진 영어 질문을 한국어로 번역하세요:
비슷한 방식으로 주어진 영어 질문을 한국어로 번역하세요:
비슷한 방식으로 주어진 영어 질문을 한국어로 번역하세요:
비슷한 방식으로 주어진 영어 질문을 한국어로 번역하세요:
비슷한 방식으로 주어진 영어 질문을 한국어로 번역하세요:
비슷한 방식으로 주어진 영어 질문을 한국어로 번역하세요:
비슷한 방식으로 주어진 영어 질문을 한국어로 번역하세요:
비슷한 방식으로 주어진 영어 질문을 한국어로 번역하세요:
비슷한 방식으로 주어진 영어 질문을 한국어로 번역하세요:
비슷한 방식으로 주어진 영어 질문을 한국어로 번역하세요:
비슷한 방식으로 주어진 영어 질문을 한국어로 번역하세요:
비슷한 방식으로 주어진 영어 질문을 한국어로 번역하세요:
비슷한 방식으로 주어진 영어 질문을 한국어로 번역하세요:
비슷한 방식으로 주어진 영어 질문을 한국어로 번역하세요:
비슷한 방식으로 주어진 영어 질문을 한국어로 번역하세요:
비슷한 방식으로 주어진 영어 질문을 한국어로 번역하세요:
비슷한 방식으로 주어진 영어 질문을 한국어로 번역하세요:
비슷한 방식으로 주어진 영어 질문을 한국어로 번역하세요:
비슷한 방식으로 주어진 영어 질문을 한국어로 번역하세요:
비슷한 방식으로 주어진 영어 질문을 한국어로 번역하세요:
비슷한 방식으로 주어진 영어 질문을 한국어로 번역하세요:
비슷한 방식으로 주어진 영어 질문을 한국어로 번역하세요:
비슷한 방식으로 주어진 영어 질문을 한국어로 번역하세요:
비슷한 

### 현재까지 만들어진 data들을 합산하여 학습 데이터로 가공함

In [ ]:
import pandas as pd
import json
import re
import glob
import os

# 1. 파일 목록 가져오기
# 예: 현재 폴더에 있는 'train'으로 시작하고 '.csv'로 끝나는 모든 파일
# 필요에 따라 경로를 수정하세요. (예: './data/*.csv')
input_file_pattern = './drive/MyDrive/Data/Break-dataset/QDMR-high-level/train_trans_decomp_*.csv'
file_list = glob.glob(input_file_pattern)

print(f"발견된 파일 목록: {file_list}")

if not file_list:
    print("처리할 파일이 없습니다. 경로와 파일명을 확인해주세요.")
    exit()

# 2. 전처리 함수 정의 (이전과 동일)
def clean_question(text):
    if pd.isna(text):
        return ""
    text = str(text).replace('[질문]', '')
    text = text.replace('\\n', ' ').replace('\n', ' ')
    return re.sub(r'\s+', ' ', text).strip()

def parse_sub_questions(text):
    if pd.isna(text):
        return []
    text = str(text).replace('\\n', ' ').replace('\n', ' ')
    parts = re.split(r'\s*\d+\.\s+', text)
    return [p.strip() for p in parts if p.strip()]

# 3. 모든 파일을 순회하며 데이터 합치기
all_result_data = []

for file_path in file_list:
    print(f"처리 중... {file_path}")
    try:
        # 파일 읽기
        df = pd.read_csv(file_path)

        # 필요한 컬럼이 있는지 확인
        if 'question_kor' not in df.columns or 'decomposition_new' not in df.columns:
            print(f"  [주의] {file_path} 파일에 필요한 컬럼이 없어 건너뜁니다.")
            continue

        # 데이터 변환 및 리스트 추가
        for _, row in df.iterrows():
            entry = {
                "question": clean_question(row.get('question_kor')),
                "sub_questions": parse_sub_questions(row.get('decomposition_new'))
            }
            all_result_data.append(entry)

    except Exception as e:
        print(f"  [에러] {file_path} 처리 중 오류 발생: {e}")

# 4. 하나의 JSON 파일로 저장
output_file = './drive/MyDrive/Data/Break-dataset/QDMR-high-level/combined_question_decomposition_1_17.json'
with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(all_result_data, f, ensure_ascii=False, indent=2)

print(f"\n총 {len(file_list)}개의 파일에서 {len(all_result_data)}개의 데이터를 변환하여")
print(f"'{output_file}' 파일로 저장했습니다.")

발견된 파일 목록: ['./drive/MyDrive/Data/Break-dataset/QDMR-high-level/train_trans_decomp_1.csv', './drive/MyDrive/Data/Break-dataset/QDMR-high-level/train_trans_decomp_2.csv', './drive/MyDrive/Data/Break-dataset/QDMR-high-level/train_trans_decomp_3_1.csv', './drive/MyDrive/Data/Break-dataset/QDMR-high-level/train_trans_decomp_3_2.csv', './drive/MyDrive/Data/Break-dataset/QDMR-high-level/train_trans_decomp_4.csv', './drive/MyDrive/Data/Break-dataset/QDMR-high-level/train_trans_decomp_5.csv', './drive/MyDrive/Data/Break-dataset/QDMR-high-level/train_trans_decomp_6.csv', './drive/MyDrive/Data/Break-dataset/QDMR-high-level/train_trans_decomp_7.csv', './drive/MyDrive/Data/Break-dataset/QDMR-high-level/train_trans_decomp_8.csv', './drive/MyDrive/Data/Break-dataset/QDMR-high-level/train_trans_decomp_9.csv', './drive/MyDrive/Data/Break-dataset/QDMR-high-level/train_trans_decomp_10.csv', './drive/MyDrive/Data/Break-dataset/QDMR-high-level/train_trans_decomp_11.csv', './drive/MyDrive/Data/Break-datase

In [ ]:
# 검증 데이터 생성
trans_decomp(data=data_decomposed, i=10500, j=11000, k=99) #gemini-2.5-flash

10500번째 질문 번역 결과 : 더 봉고 클럽은 1977년 7월 7일에 미국에서 무엇에 참여했습니까?
trans_success
10500번째 질문 분해 결과 : > 1. 미국 내에서 1977년 7월 7일에 있었던 일을 찾으세요.
2. The Bongo Club이 참여했던 것에 해당하는 것을 찾아내세요.
3. The Bongo Club이 참여했던 바로 '그것'이 무엇인지 확인하세요.
decomp_success
10501번째 질문 번역 결과 : '오페라의 유령'과 '지저스 크라이스트 슈퍼스타' 등의 유명 작품을 만든 영국의 작곡가이자 뮤지컬 기획자가 1977년에 설립한 국제 회사의 이름은 무엇입니까?
trans_success
10501번째 질문 분해 결과 : 1. 뮤지컬 '오페라의 유령(The Phantom of the Opera)'과 '지저스 크라이스트 슈퍼스타(Jesus Christ Superstar)' 등의 유명 작품을 만든 영국의 작곡가 겸 제작자를 찾습니다.
2. 1977년에 [Step 1에서 찾은 작곡가]가 설립한 국제 회사를 확인합니다.
3. [Step 2에서 확인한 회사]의 이름을 밝힙니다.
decomp_success
10502번째 질문 번역 결과 : [question]\n자크 다르크와 이자벨 로메의 딸과 관련된 박물관의 이름은 무엇입니까?
trans_success
10502번째 질문 분해 결과 : > 1. 자크 다르크(Jacques d'Arc)와 이자벨 로메(Isabelle Romée)의 딸이 누구인지 파악합니다.
> 2. 그 인물과 관련이 있는 박물관을 찾습니다.
> 3. 찾아낸 박물관의 이름을 알려줍니다.
decomp_success
10503번째 질문 번역 결과 : 두 미국 음악가인 재닛 가드너(Janet Gardner)와 마일즈 케네디(Myles Kennedy) 중 누가 더 나이가 많습니까?
trans_success
10503번째 질문 분해 결과 : 1. 재닛 가드너의 나이를 확인합니다.
2. 마일스 케네디의 나이를 확인합니다.
3. 재닛 